In [1]:
import pandas as pd
from glasspy.predict import GlassNet
from glasspy.data import SciGlass
from functools import partial
 
from funcoes import cria_populacao_compostos as cria_populacao
from funcoes import funcao_objetivo_pop as funcao_objetivo_pop
from funcoes import selecao_torneio_max as funcao_selecao
from funcoes import cruzamento_ponto_duplo as funcao_cruzamento
from funcoes import mutacao_sucessiva as funcao_mutacao_1
from funcoes import mutacao_simples as funcao_mutacao_2

C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
df_compounds_and_prices = pd.read_csv('../analise_exploratoria/Prices.csv',sep=',')

COMPOUNDS = list(df_compounds_and_prices['Oxide Coumpounds'])
NUM_COMPOUNDS = len(COMPOUNDS)
VALOR_MAX_COMPOUNDS = 100

TAMANHO_POPULACAO = 300
NUM_GERACOES = 10
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.05
CHANCE_DE_MUTACAO_POR_GENE = 0.25
TAMANHO_TORNEIO = 3

In [3]:
source = SciGlass()
model = GlassNet()

funcao_objetivo = partial(funcao_objetivo_pop, compostos=COMPOUNDS, modelo=model)

C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
populacao = cria_populacao(TAMANHO_POPULACAO, NUM_COMPOUNDS, VALOR_MAX_COMPOUNDS)

hall_da_fama = []

for n in range(NUM_GERACOES):
    print(f'Geração {n}', end='\r') 
    
    # Seleção
    fitness = funcao_objetivo(populacao)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao_1(proxima_geracao, CHANCE_DE_MUTACAO, CHANCE_DE_MUTACAO_POR_GENE, VALOR_MAX_COMPOUNDS)
    funcao_mutacao_2(proxima_geracao, CHANCE_DE_MUTACAO, VALOR_MAX_COMPOUNDS)
        
    # Atualização do hall da fama
    fitness = funcao_objetivo(proxima_geracao)
        
    maior_fitness = max(fitness)
    indice = fitness.index(maior_fitness)
    hall_da_fama.append(proxima_geracao[indice])    
    
    # Encerramento
    populacao = proxima_geracao
    
    
fitness = funcao_objetivo(hall_da_fama)
maior_fitness = max(fitness)
indice = fitness.index(maior_fitness)
melhor_individuo_observado = hall_da_fama[indice]

melhor_individuo_observado

KeyboardInterrupt: 

In [ ]:
dict_composition = dict(zip(COMPOUNDS, melhor_individuo_observado))
display(dict_composition)
predictions = model.predict(dict_composition)
predictions

{'SiO2': 59,
 'P2O5': 28,
 'ZrO2': 25,
 'Na2O': 100,
 'Al2O3': 0,
 'Fe2O3': 49,
 'CaO': 19,
 'MgO': 25,
 'MnO': 88,
 'GeO2': 15,
 'Li2O': 8,
 'Ta2O5': 76,
 'ZnO': 77,
 'SrO': 89,
 'CdO': 100,
 'SnO2': 72,
 'B2O3': 63,
 'La2O3': 24,
 'Ga2O3': 60,
 'Y2O3': 97,
 'TiO2': 85,
 'Nb2O5': 74,
 'PbO': 41,
 'HfO2': 36,
 'WO3': 30,
 'Sb2O3': 61,
 'Bi2O3': 1,
 'BaO': 82,
 'Cr2O3': 7,
 'Cu2O': 7,
 'BeO': 42,
 'CuO': 19,
 'Nd2O3': 100,
 'CeO2': 81,
 'Cs2O': 88,
 'As2O3': 5,
 'NH4NO3': 9,
 'MoO3': 55,
 'FeO': 30,
 'Mn2O3': 0,
 'SO2': 45,
 'Ag2O': 10,
 'TeO2': 97,
 'CoO': 71,
 'In2O3': 44,
 'Sc2O3': 97,
 'NiO': 27,
 'V2O5': 80,
 'Li2SO4': 33,
 'As2O5': 51,
 'MnO2': 58,
 'Sm2O3': 19,
 'Gd2O3': 86,
 'Tb2O3': 84,
 'Dy2O3': 22,
 'Ho2O3': 88,
 'Er2O3': 88,
 'Yb2O3': 100,
 'Co3O4': 46,
 'Fe3O4': 20,
 'SnO': 54,
 'Mn3O4': 60,
 'Pr2O3': 91,
 'CrO3': 74,
 'Na2SO4': 29,
 'Pr6O11': 58,
 'Co2O3': 33,
 'Ti2O3': 52,
 '(NH4)2SO4': 94,
 'K2SO4': 89,
 'Cs2SO4': 17,
 'Eu2O3': 43,
 'SeO2': 19,
 'Lu2O3': 58,
 'BiOCl': 0,

,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,...,Cp1673K,TMaxGrowthVelocity,MaxGrowthVelocity,CrystallizationPeak,CrystallizationOnset,SurfaceTensionAboveTg,SurfaceTension1173K,SurfaceTension1473K,SurfaceTension1573K,SurfaceTension1673K
0,2689.206831,1597.94443,1237.722467,981.112832,1324.628089,1441.648635,946.29137,926.809552,959.302608,1042.876223,...,1508.708161,987.098208,-1.73204,1571.381364,1692.699084,0.597647,0.42872,0.147784,0.385956,1.677151


In [ ]:
sorted(list(dict_composition.values()))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 4,
 4,
 5,
 5,
 5,
 5,
 7,
 7,
 7,
 8,
 8,
 9,
 9,
 10,
 11,
 12,
 13,
 13,
 14,
 15,
 16,
 17,
 17,
 18,
 19,
 19,
 19,
 19,
 20,
 21,
 21,
 22,
 23,
 24,
 25,
 25,
 27,
 28,
 29,
 30,
 30,
 30,
 33,
 33,
 34,
 36,
 36,
 41,
 42,
 43,
 44,
 45,
 46,
 49,
 50,
 50,
 51,
 52,
 54,
 55,
 55,
 56,
 57,
 58,
 58,
 58,
 59,
 60,
 60,
 61,
 61,
 63,
 64,
 66,
 70,
 71,
 71,
 71,
 72,
 73,
 74,
 74,
 75,
 76,
 77,
 77,
 77,
 80,
 81,
 82,
 82,
 84,
 84,
 85,
 86,
 88,
 88,
 88,
 88,
 89,
 89,
 89,
 89,
 90,
 91,
 91,
 94,
 94,
 97,
 97,
 97,
 100,
 100,
 100,
 100]

In [ ]:
float(predictions['Density293K'].iloc[0])

4.108053555358609

In [ ]:
import numpy as np

melhor_individuo_observado_relacao_grama = np.array(melhor_individuo_observado)/sum(melhor_individuo_observado) 

preco = sum(melhor_individuo_observado_relacao_grama * np.array(list(df_compounds_and_prices['Price per gram(dolar/gram)'])))
preco

13.972760677290008

In [ ]:
df_compounds_and_prices['Price per gram(dolar/gram)'].describe()

count    126.000000
mean      19.569176
std       65.880691
min        0.000037
25%        0.502725
50%        2.820000
75%       10.550000
max      593.000000
Name: Price per gram(dolar/gram), dtype: float64

In [ ]:
melhor_individuo_observado_relacao_grama

array([0.01019703, 0.00483927, 0.00432077, 0.0172831 , 0.        ,
       0.00846872, 0.00328379, 0.00432077, 0.01520913, 0.00259246,
       0.00138265, 0.01313515, 0.01330798, 0.01538196, 0.0172831 ,
       0.01244383, 0.01088835, 0.00414794, 0.01036986, 0.0167646 ,
       0.01469063, 0.01278949, 0.00708607, 0.00622191, 0.00518493,
       0.01054269, 0.00017283, 0.01417214, 0.00120982, 0.00120982,
       0.0072589 , 0.00328379, 0.0172831 , 0.01399931, 0.01520913,
       0.00086415, 0.00155548, 0.0095057 , 0.00518493, 0.        ,
       0.00777739, 0.00172831, 0.0167646 , 0.012271  , 0.00760456,
       0.0167646 , 0.00466644, 0.01382648, 0.00570342, 0.00881438,
       0.0100242 , 0.00328379, 0.01486346, 0.0145178 , 0.00380228,
       0.01520913, 0.01520913, 0.0172831 , 0.00795022, 0.00345662,
       0.00933287, 0.01036986, 0.01572762, 0.01278949, 0.0050121 ,
       0.0100242 , 0.00570342, 0.00898721, 0.01624611, 0.01538196,
       0.00293813, 0.00743173, 0.00328379, 0.0100242 , 0.     